In [16]:
import numpy as np
import string
N = 10000
alphabet = string.ascii_lowercase
D = len(alphabet)
z = np.ones(N)
RI = np.random.rand(D, N)
RI = np.where(RI>0.5, 1, -1)

def load_dataset():
    with open("names/popular-female-first.txt") as f:
        fnames = f.readlines()
    with open("names/popular-male-first.txt") as f:
        mnames = f.readlines()
    fnames = [x.lower().strip("\n") for x in fnames]
    mnames = [x.lower().strip("\n") for x in mnames]
    return fnames, mnames

def letter_combos(names, windows=[2, 3, 4]):
    letters = [set(), set(), set()]
    for name in names:
        length = len(name)
        for w in windows:
            for j in range(length-w):
                letters[w-2].add(name[j:j+w])
    letters = [list(x) for x in letters]
    return letters
#sketchy
def ngram_encode(ngram_str, letter_vecs, windows=[2,3,4]):
    vec = np.zeros(letter_vecs.shape[1])
    full_str = ngram_str
    for window in windows:
        for il, l in enumerate(full_str[:-(window-1)]):
            trivec = letter_vecs[alphabet.find(full_str[il]), :]
            for c3 in range(1, window):
                trivec = trivec * np.roll(letter_vecs[alphabet.find(full_str[il+c3]), :], c3)

            vec += trivec
    return vec

def names_vec(names):
    vec = np.zeros(RI.shape[1])
    for name in names:
        vec += ngram_encode(name, RI)
    return vec
        
def encodes(ngrams, windows=[2,3,4]):
    matrix = np.zeros((len(ngrams), N))
    for i in range(matrix.shape[0]):
        matrix[i] = ngram_encode(ngrams[i], RI, windows)
    return matrix

#complete up to additional tetragram?
def autocomplete(prefix, vec, combos, combos_vec, windows=[2,3,4]):
    length = len(prefix)
    num_suffixes = len(combos)
    for i in range(length):
        pref = ngram_encode(prefix[:i+1], RI, windows)
        est = pref * vec
        suf = np.roll(pref, i)
        for j in range(num_suffixes):
            max_i = np.argmax(combos_vec[j].dot(suf))
            print prefix[:i+1] + combos[j][max_i]

In [10]:
f, m = load_dataset()
fvec = names_vec(f)
mvec = names_vec(m)

fngram_combos = letter_combos(f)
mngram_combos = letter_combos(m)

prefix = "pax"

In [11]:
fngram_combos_vec = [encodes(x, [3, 3, 3]) for x in fngram_combos]
mngram_combos_vec = [encodes(x, [3, 3, 3]) for x in mngram_combos]
autocomplete(prefix, mvec, mngram_combos, mngram_combos_vec, [3, 3, 3])

pck
palo
pmyro
pack
paalo
pamyro
paxck
paxlor
paxrove


In [17]:
fngram_combos_vec = [encodes(x, [2, 3, 4]) for x in fngram_combos]
mngram_combos_vec = [encodes(x, [2, 3, 4]) for x in mngram_combos]
autocomplete(prefix, mvec, mngram_combos, mngram_combos_vec, [2, 3, 4])

pck
palo
pmyro
paoh
panci
papedr
paxli
paxeli
paxfeli
